# How to use NYS Data Catalog tools, SODA, and SoQL to work with large open datasets

### Using Python to submit SoQL

Especially for more involved queries, it may be easier to use a programming language to submit SoQL queries than modify a URL directly or go through the data explorer tool. We can use python's requests package with an API endpoint URL to submit queries. Below, let's calculate total Fair Fare riders by bus route and month, using a [SoQL function](https://dev.socrata.com/docs/functions/#,).

In [7]:
import requests
import pandas as pd
import io

endpoint_url = "https://data.ny.gov/resource/gxb3-akrn.csv"

soql_query = """SELECT date_trunc_ym(transit_timestamp) as month,
    bus_route,
    'Fair Fares' as fare_class,
    sum(ridership) as ridership
where fare_class_category in ('Metrocard - Fair Fare', 'OMNY - Fair Fare')
group by date_trunc_ym(transit_timestamp),
    bus_route,
    'Fair Fares'
LIMIT 50000
"""

bus_data = requests.get(
    url=endpoint_url,
    params={
        "$query" : soql_query
    }
).content

bus_data = pd.read_csv(io.StringIO(bus_data.decode("utf-8")))

bus_data

,month,bus_route,fare_class,ridership
0,2025-01-01T00:00:00.000,B1,Fair Fares,17893
1,2025-01-01T00:00:00.000,B100,Fair Fares,3223
2,2025-01-01T00:00:00.000,B101,Fair Fares,0
3,2025-01-01T00:00:00.000,B103,Fair Fares,5819
4,2025-01-01T00:00:00.000,B106,Fair Fares,0
...,...,...,...,...
3772,2025-09-01T00:00:00.000,X64,Fair Fares,0
3773,2025-09-01T00:00:00.000,X68,Fair Fares,0
3774,2025-09-01T00:00:00.000,X7,Fair Fares,0
3775,2025-09-01T00:00:00.000,X8,Fair Fares,0


In [8]:
import plotly.express as px

px.line(
    bus_data,
    x="month",
    y="ridership",
    color="bus_route"
)

This graph is not close to answering any questions about Fair Fares riders. But hopefully this demonstration can point you in some useful directions.

### Bonus

We probably won't get to this, but if you'll be programatically querying data on the NYS data catalog, you should request an app token by creating a (NYS account)[https://data.ny.gov/login]! Once you have an apptoken, you can plug it in below to make queries using the app token.

In [ ]:
import os

session = requests.Session()

# create an account and apptoken on the NYS data catalog here - https://data.ny.gov/login
session.headers.update({"access_token": os.environ["APPTOKEN"]})

query_output = session.get(
    endpoint_url, params={
        "$query" : soql_query
    }
).content

bus_data = pd.read_csv(io.StringIO(bus_data.decode("utf-8")))